EfficientV2 model test, data : https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, random_split, DataLoader
import torch
import torch.nn as nn # layer들을 호출하기 위해서
import numpy as np
import torch.optim as optim # optimization method를 사용하기 위해서
import torch.nn.init as init # weight initialization 해주기 위해서
from tqdm import tqdm

import os
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from collections import OrderedDict
from parallel import DataParallelModel,DataParallelCriterion
# %matplotlib inline

from efficientnet_pytorch import EfficientNet

In [2]:

test_set = ImageFolder(root= './stanford_only_test/Images/')
n_class = len(test_set.classes)

test_transform = transforms.Compose([
#     transforms.Resize((128, 128)), 
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

# 이미지 폴더로부터 데이터를 로드합니다.
class Dataset(Dataset):
    
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        img, label = self.ds[idx]
        if self.transform:
            img = self.transform(img)  
            return img, label

test_dataset = Dataset(test_set, test_transform)

In [3]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ModelBase(nn.Module):
    # training step
    def training_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        return loss
    
    # validation step
    def validation_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        acc = accuracy(out, targets)
        return {'val_acc':acc.detach(), 'val_loss':loss.detach()}
    
    # validation epoch end
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item()}
        
    # print result end epoch
    def epoch_end(self, epoch, result):
        print("Epoch [{}] : train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result["train_loss"], result["val_loss"], result["val_acc"]))

        
class PretrainedEfficientNet_V2(ModelBase):
    def __init__(self):
        super().__init__()
        
        self.network = EfficientNet.from_pretrained('efficientnet-b4')
#         Replace last layer
        num_ftrs = self.network._fc.in_features
        self.network._fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, n_class),
            nn.LogSoftmax(dim=1)
        )
        self.network = nn.DataParallel(self.network,device_ids=[0,1,2,3,4])
#         self.network = DataParallelModel(self.network)
        
    def forward(self, xb):
        return self.network(xb)

In [4]:
model = PretrainedEfficientNet_V2()

Loaded pretrained weights for efficientnet-b4


In [5]:
model.load_state_dict(torch.load('./models/dataloader-2_EfficV2_Acc90.pt'), strict=False)

<All keys matched successfully>

In [6]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    else:
        return data.to(device, non_blocking=True)

device = torch.device('cuda')
to_device(model, device);

In [7]:
class Dataset(Dataset):
    
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        img, label = self.ds[idx]
        if self.transform:
            img = self.transform(img)  
            return img, label
        
class DeviceDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)


test_dataset = Dataset(test_set, test_transform)
test_loader = DataLoader(test_dataset, 128*2, num_workers=4, pin_memory=True)
test_dl = DeviceDataLoader(test_loader, device)



In [8]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [9]:
evaluate(model, test_dl) 

{'val_loss': 0.41379788517951965, 'val_acc': 0.8807060122489929}